# Rolling Planning mit Flow Time

In [ ]:
from configs.config import get_path
import src.utils.converter as convert
import src.utils.presenter as show
import src.utils.checker as check

import src.utils.gen_jobs as gen_jobs
import src.utils.rolling_scheduling as rolling_schedule

import time
import pandas as pd
pd.set_option('display.max_rows', 26)

In [ ]:
import src.models.lp.flowtime_solver as solver
from src.simulation.ProductionDaySimulation import ProductionDaySimulation

In [ ]:
max_time = 60*60*2 # 2h

## Generierung des Datensatzes für 10 Tage (Ankunft)

In [ ]:
basic_data_path = get_path("data", "basic")

df_instance = pd.read_csv(basic_data_path / "00_instance.csv")
df_jssp, df_jobs_arrivals = gen_jobs.create_jobs_for_days(df_instance, day_count = 10, u_b_mmax= 0.92, shuffle = True)
df_jobs_arrivals

In [ ]:
df_jssp

## Params

In [ ]:
# fix
day_length = 1440
horizon_days = 2

In [ ]:
# init
day_numb = 0

day_start = 0
day_end = 0
planning_end = 0

# Initialisierung (Tag 0)

In [ ]:
day_numb = 0

In [ ]:
day_start = day_length*day_numb                      # 0
day_end = day_start + day_length                     # 1440
planning_end =  day_start + horizon_days*day_length  # 4320

In [ ]:
# I)
df_jssp_curr, df_times_curr = rolling_schedule.filter_jobs_by_arrival_window(df_jobs_arrivals, df_jssp, day_start, planning_end)
df_jssp_curr

In [ ]:
df_times_curr 

### Scheduling

In [ ]:
starting_time = time.time()

# Scheduling
df_plan = solver.solve_jssp_flowtime(df_jssp_curr, df_times_curr, solver= "HiGHS", msg=False, timeLimit=max_time, gapRel= 0.02)
df_plan

# Informationen
ending_time = time.time()
solver_duration = ending_time - starting_time
print(f"\nScheduling-Dauer: {int(solver_duration // 60)} Minuten und {(solver_duration % 60):.2f} Sekunden.")
df_plan

In [ ]:
show.plot_gantt_jobs(df_plan)

check.is_machine_conflict_free(df_plan)
check.is_operation_sequence_correct(df_plan)
check.is_job_timing_correct(df_plan)
check.is_start_correct(df_plan)

### Simulation

In [ ]:
simulation = ProductionDaySimulation(df_plan, vc=0.25)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)

In [ ]:
if not df_execution.empty:
    show.plot_gantt_machines(df_execution, duration_column="Simulated Processing Time")
else:
    print(f"Nothing executed on day {day_numb}")

In [ ]:
df_undone

#### Export

In [ ]:
export_data_path = get_path("data", "lp_flowtime")

In [ ]:
df_jssp.to_csv(export_data_path / "02_flowtime_all_jssp.csv", index = False)
df_jobs_arrivals.to_csv(export_data_path / "02_flowtime_all_times.csv", index = False)

In [ ]:
df_plan.to_csv(export_data_path / "02_flowtime_init_plan.csv", index = False)
df_execution.to_csv(export_data_path / "02_flowtime_execution.csv", index = False)
df_undone.to_csv(export_data_path / "02_flowtime_init_undone.csv", index = False)